In [50]:
import requests  
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import mysql.connector as SQLC
import mysql.connector
from datetime import date
import time
from bs4 import BeautifulSoup
from Bio.SeqFeature import CompoundLocation
from datetime import datetime

In [51]:
#Extrair ids genebank:

data_e_hora_atuais = datetime.now()

query= input('escolha o que quer pesquisar: ')


try:
    def url_get(i):
        url_list_id=[]
        url = f"https://www.ncbi.nlm.nih.gov/gene/?term={i}"
        url_list_id.append(url)
        return url_list_id
    
    url_get(query)

    content = []
    for url in url_get(query):
        r = requests.get(url)
        content.append(r.content)

    for c in content:
        soup = BeautifulSoup(c, 'html.parser')
        a= soup.get_text()

    existe = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

    c= ', '.join(existe)
    h= c.replace('ID: ','')
    IDS= h.split(', ')

    n_gene= IDS[0:1+(int(input('escolha o nº de genes que quer obter (máximo 20): ')))]

    n_genes = []
    seen = set()
    for item in n_gene:
        if item not in seen:
            seen.add(item)
            n_genes.append(item)

    numero_genes= len(n_genes)

    if n_genes == ['']:
        print()
        print('0 resultados para a sua pesquisa, pesquise de novo.')

except:
    ('0 resultados para a sua pesquisa, pesquise de novo.')

escolha o que quer pesquisar: diabetes
escolha o nº de genes que quer obter (máximo 20): 20


In [52]:
#Extrair ids ncbi:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
    
if Ids == ['']:
    print('0 resultados para a sua pesquisa, pesquise de novo.')
        
#Extrair description NCBI:
description=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    description.append(info.description)

#organismos
Organismos=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Organismos.append(info.annotations['organism'])
    
#SEQ
seqss=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    seqs= (f'{info.seq[0:10]}...{info.seq[-10:]}')
    seqss.append(seqs)

#percentagem de nucle:
Adenina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    c=info.seq
    count= c.count('A')
    perc= int(count/len(c)*100)
    Adenina.append(perc)

Citosina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    c=info.seq
    count= c.count('C')
    perc= int(count/len(c)*100)
    Citosina.append(perc)

Guanina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    c=info.seq
    count= c.count('G')
    perc= int(count/len(c)*100)
    Guanina.append(perc)

Timina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    c=info.seq
    count= c.count('T')
    perc= int(count/len(c)*100)
    Timina.append(perc)
    
#data
dates=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    dates.append(info.annotations['date'])

#len(SEQ)
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
tam=[]
for info in records:
    tam.append(len(info.seq))
    
    # buscar info pubmed
try:
    def url_get_id(i):
        url_list= [ ]
        for id in i:
            url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id )
            url_list.append(url)
        return url_list
    link_genebank= url_get_id(Ids)
    #print(link_genebank)

    content_id = []
    for url in url_get_id(Ids):
        r_id = requests.get(url)
        content_id.append(r_id.content)

    from bs4 import BeautifulSoup
    listas=[]

    for c in content_id:
        soup_id = BeautifulSoup(c, 'html.parser')

        lines = soup_id.find_all('meta', {'name':"ncbi_uidlist"} )

        id = ""
        url = ""
        for line in lines:
            if 'content' in line.attrs:
                id = line.attrs['content']
        if id:
            url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id)
        r2 = requests.get(url)
        r4= r2.content.decode('utf-8')
        listas.append(r4)

    cc= ', '.join(listas)
    er= cc.replace('//','')
    final= er.split(', ')

    #criar dicionário de ids ncbi e ids pubmed
    output_dict = {}
    for x in final:
        version = re.search(r'VERSION\s+(.*?)\s', x)
        pubmed = re.search(r'PUBMED\s+(.*?)\s', x)
        if version:
            versionf=version.group(1)
            output_dict.setdefault(version.group(1), [])
        if pubmed:
            output_dict[versionf].append(pubmed.group(1))
except:
    print()
    print("0 resultados na Pubmed em realção à sua query")
    
#Criar Listas só com ids ncbi e ids pubmed
id_ncbii = []
ID_PUB = []
for key, vals in output_dict.items():
    if vals:
        for val in vals:
            id_ncbii.append(key)
            ID_PUB.append(val)
    else:
        id_ncbii.append(key)
        ID_PUB.append("N/A")

new_list_ = []
seen = set()
for item in ID_PUB:
    if item not in seen:
        seen.add(item)
        new_list_.append(item)

print("Concluido")

Concluido


In [53]:
#Extrair informação de artigos
try:
    titles=[]
    authors=[]
    source=[]
    affiliation=[]
    database = 'PubMed'
    email= 'rodrigoce9@gmail.com'
    idlist= new_list_
    counter = 0
    for i in idlist:
        if i!= "N/A":
            handle = Entrez.efetch(db=database, id=i, rettype="medline", retmode="text") 
            records = Medline.parse(handle)
            for info in records:
                titles.append(info.get("TI", ["N/A"]))
                authors_string = info.get("AU", ["N/A"])
                if len(authors_string) > 5:
                    authors_h = authors_string[0:5]
                    authors.append(authors_h)
                else:
                    authors.append(authors_string) 
                source.append(info.get("SO", ["N/A"]))
                affiliation_string= info.get("AD", ["N/A"])
                if len(affiliation_string) > 5:
                    affiliation_h = affiliation_string[0:5]
                    affiliation.append(affiliation_h)
                else:
                    affiliation.append(affiliation_string)
                counter += 1
        else:
            titles.append(["N/A"])
            authors.append(["N/A"])
            source.append(["N/A"])
            affiliation.append(["N/A"])
            counter += 1
except:
    print('artigos não encontrados por um possivel bug de id')
    
#agrupar titles
titles = [ [title] for title in titles]

# agrupar dois
doi_list = []
for x in source:
    match = re.search("doi: (.*)", str(x))
    if match:
        doi_list.append(match.group(1))
    else:
        doi_list.append("N/A")
        
#agrupar authors
id_authors_dict = {i: authors[counter] if i != "N/A" else ["N/A"] for counter, i in enumerate(idlist)}        
        
pubmed_list = []
authors_list = []
for key, vals in id_authors_dict.items():
    if vals:
        for val in vals:
            pubmed_list.append(key)
            authors_list.append(val)

new_list_authors= []
seen = set()
for item in authors_list:
    if item not in seen:
        seen.add(item)
        new_list_authors.append(item)

#agrupar affi
single_affiliation_list = []
for i in affiliation:
    single_affiliation_list.extend(i)
    

id_affiliation_dict = {i: [single_affiliation_list[counter]] if i != "N/A" else ["N/A"] for counter, i in enumerate(idlist)}

pubmed_affi_list = []
affi_list = []
for key, vals in id_affiliation_dict.items():
    if vals:
        for val in vals:
            pubmed_affi_list.append(key)
            affi_list.append(val)
    else:
        pubmed_affi_list.append(key)
        affi_list.append(["N/A"])

new_list_affi= []
seen = set()
for item in single_affiliation_list :
    if item not in seen:
        seen.add(item)
        new_list_affi.append(item)

def url_get_id_p(string):
    url_list_p=[ ]
    url_id_p= "https://www.uniprot.org/uniprotkb/{}/entry".format(str(string))
    url_list_p.append(url_id_p)
    return ''.join(url_list_p)

# protein_id - PROT_ID 
list_pro=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    for i in info.features:
        if i.type == "CDS":
            pro= str(i.qualifiers["protein_id"])
            list_pro.append(pro)
div= ', '.join(list_pro)
h= div.replace("['",'')
hh= h.replace("']",'')
ID_PROT= hh.split(', ')

#result_dic = {Ids, ID_PROT} - 
result_dict = {}
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist=Ids
for ids in idlist:
    list_pro=[]
#     print( ids)
    handle = Entrez.efetch(db=database, id=ids, rettype="gb") 
    records = list(SeqIO.parse(handle,"gb"))
    handle.close()
    for info in records:
        list_pro.append(info.id)
        for i in info.features:
            if i.type == "CDS":
                pro= str(i.qualifiers["protein_id"])
                list_pro.append(pro)
        if len(list_pro)==1: #if no protein_id was found
            result_dict[info.id] = "N/A_CDS"
        else:
            list_pro = [x.replace("['",'').replace("']",'') for x in list_pro]
            result_dict[info.id] = ', '.join(list_pro[1:])
#key: id_genebank; values: ID_PROT

def dict_to_list(d, delimiter=','): #transforms dictionary to list
    new_list = []
    for k, v in d.items():
        if isinstance(v, str):
            values = v.split(delimiter)
            for val in values:
                new_list.append([k, val.strip()])
        else:
            new_list.append([k, v])
    return new_list
dict_list=dict_to_list(result_dict)


from collections import OrderedDict
def get_CDS_info(result_dict): #gets info for CDS [ID_CDS, location, Translation]
    database = 'nucleotide'
    email= 'rodrigoce9@gmail.com'
    Entrez.email = email
    cds_location_list = []
    several_location = []
    record_types={}
    processed_i = set()
    for i, value in result_dict.items():
        if i in processed_i:
            continue
        if value == 'N/A_CDS':
            cds_location = ['N/A_CDS', 'N/A', 'N/A']
            cds_location_list.append(cds_location)
        else:
            handle = Entrez.efetch(db=database, id=i, rettype="gb") 
            records = list(SeqIO.parse(handle,"gb"))
            handle.close()
            for info in records:
                for i in info.features:
                    if i.type == "CDS":
                        cds_location = []
                        i_d = str(i.qualifiers["protein_id"])
                        translation =  str(i.qualifiers["translation"])
                        cds_location.append(i_d)
                        if isinstance(i.location, CompoundLocation):
                            for sub_location in i.location.parts:
                                several_location.append("[{} : {}]".format(sub_location.start, sub_location.end))
                            cds_location.append(several_location)
                            cds_location.append(translation)
                        else:
                            cds_location.append("[{} : {}]".format(i.location.start, i.location.end))
                            cds_location.append(translation)
                        cds_location_list.append(cds_location)      
                processed_i.add(i)
            handle.close()
    return cds_location_list   

WEBSITE_API = "https://rest.uniprot.org"
fields = ["accession","organism_name","protein_name","cc_subcellular_location","cc_function", "sequence"]
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs);
    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()
    return response

#get list of [ID_Uniprot, Subcellular location, Function, Sequence, Sequence length]
def get_field_for_id(ID_PROT, field): 
    response = get_url("{}/uniprotkb/search?query={}&fields={}&size=1&format=tsv".format(WEBSITE_API,ID_PROT,field))
    return str(response.content)

try:
    def get_list_uniprot(ID_PROT, result_dict):
        results = []
        result = []
        tmp= []
        easy=dict_to_list(result_dict)
        for first_index, first_value in easy:
            tmp= []
            if first_value != 'N/A_CDS':
                for field in fields:
                    result = get_field_for_id(first_value, field)
                    tmp.append(result)
            else:
                result = ['N/A_Uniprot']
                tmp.append(result)
            results.append(tmp)   
        uniprot_final_list=[]
        for index, values in enumerate(results):
            uniprot_list=[]
            n_a = re.search(r'(N/A_Uniprot)', str(values), re.DOTALL)
            if len (values)==1:
                if n_a:
                    uniprot_list.append(n_a.group(1))
                    tmp_list='N/A','N/A','N/A','N/A'
                    uniprot_list.extend(tmp_list)
            else:
                for i in values:
                    entry = re.search(r'b\'Entry\\n(.+?(?=\\n\'))', str(i), re.DOTALL)
                    function = re.match( r'b\'Function \[CC\]\\n.{9} (.+?(?=\\n\'))', str(i), re.DOTALL )
                    location_exist = re.search( r'b\'Subcellular location \[CC\]\\nSUBCELLULAR LOCATION: (.+?(?=\\n\'))', str(i), re.DOTALL )
                    location_notexist = re.search( r'b\'Subcellular location \[CC\]\\n\\n\'',str(i), re.DOTALL )   
                    sequence = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', str(i), re.DOTALL)
                    if entry:
                        ent=entry.group(1)
                        uniprot_list.append(entry.group(1))            
                    if function:
                        uniprot_list.append(function.group(1))
                    if location_exist:
                        uniprot_list.append(location_exist.group(1))
                    if location_notexist:
                        uniprot_list.append("N/A")
                    if sequence:
                        uniprot_list.append(sequence.group(1))
                        uniprot_list.append(len(sequence.group(1)))
                        uniprot_list.append(url_get_id_p(ent))

            if len(uniprot_list) < 6: #sometimes, there is no function associated
                uniprot_list.insert(2, 'N/A')
            uniprot_final_list.append(uniprot_list)

        return uniprot_final_list
    get_Uniprot=get_list_uniprot(ID_PROT,result_dict)    
except:
    print("A informação foi apagada ")
    
#[idgenebank, protein_id, id do uniprot] 
#joins to final list [idgenebank, ID_PRO, ID_Uniprot]
def join_ids_CDS(lista, uniprotID): 
    join_list_all = []
    for key, value in lista:
        join_list = []
        join_list.append(key)
        join_list.append(value)
        join_list_all.append(join_list) 
    for index, values in enumerate(uniprotID):
        join_list_all[index].append(uniprotID[index][0])
    return join_list_all
join_ids=join_ids_CDS(dict_list,get_list_uniprot(ID_PROT, result_dict))


def join_lists(list1, list2): #joins to final list [idgenebank, ID_PRO, ID_Uniprot,location, Translation]
    final_result =[]
    list_tmp=[]
    tmp=''
    for item, value in enumerate(list1):
        result = []
        string_item = str(list2[item][0])
        string_item = string_item.replace("[","").replace("]","").replace("'","")
        if list1[item][1] == string_item:
            result.append(list1[item] + list2[item][1:])
            tmp=list1[item][0]
        elif list1[item][0] == tmp:
            list_tmp = []
            list_tmp.append(tmp)
            list_tmp.append(list1[item][1])
            list_tmp.append(list1[item][2])
            result.append(list_tmp + list2[item][1:])
        else:
            result.append(list1[item])
        final_result.extend(result)
    return final_result
join_CDS=join_lists(join_ids, get_CDS_info(result_dict))

def count_(genes):
    return len(genes)
count_(Ids)
count_(ID_PROT)



print("Concluido")

Concluido


In [56]:
#delete de tudo 

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
        
        a= "delete from PubMed_Affiliation"
        Z= "delete from PubMed_Authors"
        b= "delete from Affiliation"
        c= "delete from Authors"
        d= "delete from gene_PubMed"
        e= "delete from PubMed"
        f= "delete from CDS"
        g= "delete from Uniprot"
        h= "delete from Gene"
        #i= "delete from History"     
        
        Cursor.execute(a)
        Cursor.execute(Z)
        Cursor.execute(b)
        Cursor.execute(c)
        Cursor.execute(d)
        Cursor.execute(e)
        Cursor.execute(f)
        Cursor.execute(g)
        Cursor.execute(h)
        #Cursor.execute(i)
        
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [57]:
#Povoação "History"

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql1= "INSERT INTO History (search, Genes_number_input, Day, Genes_number_NCBI, Protein_number  )   VALUES (%s, %s, %s, %s, %s)"
    val1=(query, numero_genes, data_e_hora_atuais, count_(Ids), count_(ID_PROT) )
    Cursor.execute(sql1,val1)
    
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) ) 
    
search_id=[]
try:
    sql2= "Select ID_search FROM History"
    Cursor.execute(sql2)
    for row in Cursor:
        search_id.append(str(row)) 
        
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
div= ', '.join(search_id)
h= div.replace("(",'')
hh= h.replace(",)",'')
SEARCH_ID= hh.split(', ')
Hist= SEARCH_ID[-1]

#Povoação "Gene"-

des = (description)

try:
    for index, value in enumerate(Ids):
       
        sql3= "INSERT INTO Gene (ID_genebank, Description, Organism, sequence, Date_publish, length, Adenina, Citosina, Guanina, Timina, Link, ID_search) VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val3=(value, des[index], Organismos[index], seqss[index], dates[index], tam[index], Adenina[index], Citosina[index], Guanina[index], Timina[index], link_genebank[index], Hist)

        Cursor.execute(sql3,val3)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
#Povoar "PubMed"

try:
    for index, value in enumerate(new_list_):
        sql4= "INSERT INTO PubMed (ID_PumMed, title, Doi_number) VALUES (%s, %s, %s)"
        val4=(str(new_list_[index]), str(titles[index]), str(doi_list[index]) )
        Cursor.execute(sql4,val4)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    

    
#retirar os valores AI de pubmed
ID_AI=[]

try:
    sql5= "Select ID_AI_PubMed FROM PubMed"
    Cursor.execute(sql5)
    for row in Cursor:
        ID_AI.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
      
div_= ', '.join(ID_AI)
h_= div_.replace("(",'')
hh_= h_.replace(",)",'')
SEARCH_ID_= hh_.split(', ')
#print(SEARCH_ID_)

#Povação "Gene-Pubmed"
number_map = {}
next_number = int(SEARCH_ID_[0])
relation = []
for number in ID_PUB:
    if number not in number_map:
        number_map[number] = next_number
        next_number += 1
    relation.append(number_map[number])
    
try:
    for index, value in enumerate(id_ncbii):
        
        sql6= "INSERT INTO gene_PubMed (ID_genebank, ID_AI_PubMed) VALUES (%s, %s)"
        val6=(id_ncbii[index], relation[index])
    
        Cursor.execute(sql6,val6)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
#Povação "authors"
try:
    for index, value in enumerate(new_list_authors):
        
        sql7= "INSERT INTO Authors (Name) VALUES (%s)"
        val7=(new_list_authors[index],)
    
        Cursor.execute(sql7,val7)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    

#retirar os valores AI de authors
ID_AI_Authors=[]

try:
    sql8= "Select ID_Authors FROM Authors"
    Cursor.execute(sql8)
    for row in Cursor:
        ID_AI_Authors.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
div_Authors= ', '.join(ID_AI_Authors)
h_Authors= div_Authors.replace("(",'')
hh_Authors= h_Authors.replace(",)",'')
SEARCH_ID_Authors= hh_Authors.split(', ')
#print(SEARCH_ID_Authors)

#Povação "Pubmed-Authors"
number_map_ = {}
next_number = int(SEARCH_ID_Authors[0])
output_authors = []
for number in authors_list:
    if number not in number_map_:
        number_map_[number] = next_number
        next_number += 1
    output_authors.append(number_map_[number])

number_map_pub = {}
next_number = int(SEARCH_ID_[0])
output_pub = []
for number in pubmed_list:
    if number not in number_map_pub:
        number_map_pub[number] = next_number
        next_number += 1
    output_pub.append(number_map_pub[number])

try:
    for index, value in enumerate(output_pub):
        
        sql9= "INSERT INTO PubMed_Authors (ID_AI_PubMed, ID_Authors) VALUES (%s, %s)"
        val9=(str(output_pub[index]),str(output_authors[index]))
    
        Cursor.execute(sql9,val9)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
#Povação "affi"

try:
    for index, value in enumerate(new_list_affi):
        
        sql10= "INSERT INTO Affiliation (Info) VALUES (%s)"
        val10=(new_list_affi[index],)
    
        Cursor.execute(sql10,val10)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
#retirar os valores AI de affi
ID_AI_Affiliation=[]

try:
    sql11= "Select ID_Affiliation FROM Affiliation"
    Cursor.execute(sql11)
    for row in Cursor:
        ID_AI_Affiliation.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
div_Affiliation= ', '.join(ID_AI_Affiliation)
h_Affiliation= div_Affiliation.replace("(",'')
hh_Affiliation= h_Affiliation.replace(",)",'')
SEARCH_ID_Affiliation= hh_Affiliation.split(', ')
#print(SEARCH_ID_Affiliation)

#Povação "Pubmed-Affiliation"
number_map_affi_pub = {}
next_number = int(SEARCH_ID_[0])
output_pub_affi = []
for number in pubmed_affi_list:
    if number not in number_map_affi_pub:
        number_map_affi_pub[number] = next_number
        next_number += 1
    output_pub_affi.append(number_map_affi_pub[number])
    
number_map_affi = {}
next_number = int(SEARCH_ID_Affiliation[0])
output_affi = []
for number in affi_list:
    if number not in number_map_affi:
        number_map_affi[number] = next_number
        next_number += 1
    output_affi.append(number_map_affi[number])

try:
    for index, value in enumerate(output_pub_affi):
        
        sql12= "INSERT INTO PubMed_Affiliation (ID_AI_PubMed, ID_Affiliation) VALUES (%s, %s)"
        val12=(str(output_pub_affi[index]),str(output_affi[index]))
    
        Cursor.execute(sql12,val12)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    

#Povoação "Uniprot" 
# print(get_Uniprot) #(ID_Uniprot, Subcelular location, Function, Protein seq, length_aa) 

a=["N/A_Uniprot"]

try:
    sec1 = 'INSERT INTO Uniprot (ID_Uniprot, Subcelular_Location, Function, Protein_sequence, length_aa, Link_Uniprot) VALUES (%s, %s, %s, %s, %s, %s)'
    Cursor.execute(sec1, (a[0],a[0],a[0],a[0],a[0],a[0]))
    for index, value in enumerate(get_Uniprot):
        if str(value[0]) == 'N/A_Uniprot':
            continue
        else:
            sql13= "INSERT INTO Uniprot (ID_Uniprot, Subcelular_Location, Function, Protein_sequence, length_aa, Link_Uniprot) VALUES (%s, %s, %s, %s, %s, %s)"
            val13=(str(value[0]), str(value[1]), str(value[2]), str(value[3]), str(value[4]), str(value[5]))
            Cursor.execute(sql13,val13)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    

#Povoação "CDS"
try:
    for item in join_CDS:
        sql14= "INSERT INTO CDS (ID_CDS, Translation, Location, ID_genebank, ID_Uniprot) VALUES (%s, %s, %s, %s, %s)"
        val14=(str(item[1]),str(item[4]),str(item[3]),str(item[0]),str(item[2]))
        Cursor.execute(sql14,val14)
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

print("Concluido")

Concluido


In [44]:
import pandas as pd 

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
tabela1 = pd.read_sql("Select * FROM History", DataBase)
print(tabela1)
tabela2 = pd.read_sql("Select * FROM Gene", DataBase)
print(tabela2)
tabela3 = pd.read_sql("Select * FROM gene_PubMed", DataBase)
print(tabela3)
tabela4 = pd.read_sql("Select * FROM PubMed", DataBase)
print(tabela4)
tabela5 = pd.read_sql("Select * FROM PubMed_Authors", DataBase)
print(tabela5)
tabela6 = pd.read_sql("Select * FROM Authors", DataBase)
print(tabela6)
tabela7 = pd.read_sql("Select * FROM PubMed", DataBase)
print(tabela7)
tabela8 = pd.read_sql("Select * FROM PubMed_Affiliation", DataBase)
print(tabela8)
tabela9 = pd.read_sql("Select * FROM Affiliation", DataBase)
print(tabela9)
tabela10 = pd.read_sql("Select * FROM CDS", DataBase)#print(tabela10)
print(tabela10)
tabela11 = pd.read_sql("Select * FROM Uniprot", DataBase)
print(tabela11)
DataBase.close()

C:\Users\rodri\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\rodri\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\rodri\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\rodri\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqli

   ID_search     Search  Genes_number_input                 Day  \
0         22  diabetes                   15 2023-01-19 11:54:41   

   Genes_number_NCBI  Protein_number  
0                  8               4  
  ID_genebank                                        Description  \
0    M90830.1  Xerocomus chrysenteron 18S ribosomal RNA gene,...   
1    M94340.1  Xerocomus chrysenteron 18S ribosomal RNA gene,...   
2    V00883.1  Rabbit (O. cuniculus) germ line gene coding fo...   
3    V01310.1  Yeast (S. cerevisiae) gene HIS4 (histidine met...   
4    X61243.1                           C.elegans repetitive DNA   
5    X67017.1  S.cerevisiae cyt2 gene for cytochrome c heme l...   
6    Z15032.1  S.cerevisiae gene for S.pombe cdc21+ homologue...   
7    Z17674.1  ATTS0248 Gif-SeedA+B Arabidopsis thaliana cDNA...   

                    Organism date_publish                 sequence  length  \
0  Xerocomellus chrysenteron  09-APR-2001  GCAAGTCTGG...TTGACGGAAG     604   
1  Xerocomellus ch

C:\Users\rodri\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\rodri\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\rodri\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\rodri\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqli

       ID_CDS                                        Translation  \
0  CAA24252.1  ['MVHFTAEEKAAITSTWKLVDVEDAGAEALGRLLVVYPWTQRFFD...   
1  CAA24617.1  ['MVLPILPLIDDLASWNSKKEYVSLVGQVLLDGSSLSNEEILQFS...   
2  CAA47407.1  ['MMSSDQQGKCPVDEETKKLWLREHGNEAHPGATAPGNQLECSAN...   
3  CAA78750.1  ['STSKSQILQYVHKITPRGVYTSGKGSSAVGLTAYITRDVDTKQL...   
4     N/A_CDS                                                N/A   
5     N/A_CDS                                                N/A   
6     N/A_CDS                                                N/A   
7     N/A_CDS                                                N/A   

                                          Location ID_genebank   ID_Uniprot  
0  ['[223 : 316]', '[440 : 662]', '[1479 : 1608]']    V00883.1       P02099  
1                                    [1331 : 3731]    V01310.1       P00815  
2                                      [246 : 921]    X67017.1       Q00873  
3                                      [<0 : >228]    Z15032.1       P30665

C:\Users\rodri\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [22]:
#delete de tudo 

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
        
        a= "delete from PubMed_Affiliation"
        Z= "delete from PubMed_Authors"
        b= "delete from Affiliation"
        c= "delete from Authors"
        d= "delete from gene_PubMed"
        e= "delete from PubMed"
        f= "delete from CDS"
        g= "delete from Uniprot"
        h= "delete from Gene"
        i= "delete from History"     
        
        Cursor.execute(a)
        Cursor.execute(Z)
        Cursor.execute(b)
        Cursor.execute(c)
        Cursor.execute(d)
        Cursor.execute(e)
        Cursor.execute(f)
        Cursor.execute(g)
        Cursor.execute(h)
        Cursor.execute(i)
        
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()